In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [3]:
from __future__ import nested_scopes

In [4]:
import re
import os
#import pandas
#pandas.set_option('display.max_rows', None)


In [5]:
import findspark
findspark.init("/home/xgboost/spark-3.0.0-bin-hadoop2.7")

In [6]:
import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, DateType,
    TimestampType, StringType, LongType, IntegerType, DoubleType,FloatType)
from pyspark.sql.functions import to_date, floor
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import lit
import time, timeit
from pyspark.storagelevel import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

In [7]:
from pyspark.ml import Pipeline
import pandas
import numpy as np

In [8]:
import math
from functools import reduce
import json

In [9]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from datetime import date
from datetime import datetime

In [10]:

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/xgboost/xgboost4j/jars/xgboost4j_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/xgboost4j-spark_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/spark-arrow-datasource-0.9.0-jar-with-dependencies.jar pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/yuzhou/dev_spark/xgboost_hongze/baseline/xgboost4j_2.12-1.0.0-SNAPSHOT.jar,/home/yuzhou/dev_spark/xgboost_hongze/baseline/xgboost4j-spark_2.12-1.0.0-SNAPSHOT.jar pyspark-shell'
os.environ['ARROW_LIBHDFS3_DIR'] = '/home/xgboost/miniconda3/lib'
os.environ['LD_LIBRARY_PATH'] = '/home/xgboost/miniconda3/lib'

In [11]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [12]:
arrow_datasource_jar = '/home/xgboost/xgboost4j/jars/spark-arrow-datasource-0.9.0-jar-with-dependencies.jar'

def create_cntx(executors_per_node,cores_per_executor):
    cache_size=30000
    total_size=340000
    print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size/(nodes*executors_per_node))),int(math.floor(cache_size/(nodes*executors_per_node)))))
    conf = SparkConf()\
        .set('spark.default.parallelism', '1')\
        .set('spark.executor.instances', '{:d}'.format(executors_per_node*nodes))\
        .set('spark.files.maxPartitionBytes', '256m')\
        .set('spark.app.name', 'test')\
        .set('spark.rdd.compress', 'False')\
        .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')\
        .set('spark.executor.cores','{:d}'.format(cores_per_executor))\
        .set('spark.executor.memory', '{:d}m'.format(int(math.floor(total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size/(nodes*executors_per_node)))))\
        .set('spark.task.cpus','1')\
        .set('spark.driver.memory','24g')\
        .set('spark.memory.offHeap.enabled','True')\
        .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size/(nodes*executors_per_node)))))\
        .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size/(nodes*executors_per_node)))+3000))\
        .set('spark.sql.join.preferSortMergeJoin','False')\
        .set('spark.memory.storageFraction','0.5')\
        .set('spark.executor.extraJavaOptions',\
            '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails')\
        .set('spark.driver.maxResultSize', 0)\
        .set('spark.eventLog.enabled', 'True') \
        .set('spark.resourceProfile.executorReuse', 'True')\
        .set('spark.dynamicAllocation.enabled', 'True')\
        .set('spark.shuffle.service.enabled', 'False')\
        .set('spark.dynamicAllocation.shuffleTracking.enabled', 'True')\
        .set('spark.executorEnv.ARROW_LIBHDFS3_DIR', '/home/xgboost/miniconda3/lib')\
        .set('spark.executorEnv.LD_LIBRARY_PATH', '/home/xgboost/miniconda3/lib')\
        .set('spark.driver.extraClassPath', arrow_datasource_jar) \
        .set('spark.executor.extraClassPath', arrow_datasource_jar) \
        .set('spark.files', arrow_datasource_jar) \
        .set('spark.eventLog.dir','file:///home/xgboost/spark_events')\
        .set('spark.dynamicAllocation.exeutorIdleTimeout','10000')\
        .set('spark.dynamicAllocation.minExecutors', '{:d}'.format(executors_per_node*nodes))\
        .set('spark.dynamicAllocation.maxExecutors', '{:d}'.format(executors_per_node*nodes))\
        .set('spark.dynamicAllocation.initialExecutors', '{:d}'.format(executors_per_node*nodes))\
        .set('xgboost.spark.arrow.optimization.enabled', 'True') \
        .setAppName('spark-xgboost')

    
    spark = SparkSession.builder\
                .master("spark://sr572:7077")\
                .config(conf=conf)\
                .getOrCreate()
        
    sc = spark.sparkContext
    sc.setLogLevel('INFO')
    sc.addPyFile('/home/xgboost/xgboost4j/sparkxgb_1.23.zip')
    return sc, spark

In [13]:
def load_bitdata_arrow_flattern(spark, path):
    # spark.setConf('spark.sql.files.maxPartitionBytes', '1024')
    #df = spark.read.format('arrow').load('hdfs://sr507/user/yuzhou/xgboost_36_files_double_label.dataframe.parquet')
  
    df = spark.read.format('arrow').load(path)
    #df = spark.read.parquet(path)
    print("DataFrame schma: ", df.schema)
    #df.groupBy(F.spark_partition_id()).count().show(300)
    return df

In [14]:
def run_train(train_data, params):
    t1 = timeit.default_timer()
    print("{} : {}".format("Start time", t1))
    xgboost =  XGBoostClassifier(**params)
    # xgboost.setNumClass(3)
    model = xgboost.fit(train_data)
    t2 = timeit.default_timer()
    train_time = t2 - t1
    return model, train_time

In [15]:
SPARK_HOME='/home/xgboost/spark-3.0.0-bin-hadoop2.7'
!$SPARK_HOME/sbin/start-master.sh
!$SPARK_HOME/sbin/start-slave.sh spark://sr572:7077 -c 8

starting org.apache.spark.deploy.master.Master, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.master.Master-1-sr572.out
starting org.apache.spark.deploy.worker.Worker, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.worker.Worker-1-sr572.out


# Standalone Delpoy Test

In [16]:
nodes=1
executors_per_node=1
cores_per_executor=8

overall_start_time = timeit.default_timer()
params = {  'labelCol': "delinquency_12",
            'numRound': 100,
            'maxDepth': 8,
            'maxLeaves': 256,
          'alpha': 0.9,
          'eta': 0.1,
          'gamma': 0.1,
          'subsample': 1.0,
          'reg_lambda': 1.0,
          'scalePosWeight': 2.0,
          'minChildWeight': 30.0,
          'treeMethod': 'hist',
          'objective': 'reg:squarederror', #if xgboost v0.82 needs to use 'reg:linear'. If >= 0.9, uses squarederror
          'growPolicy': 'lossguide',  #depthwise
          'numWorkers': 1, # the number of executor number
          'nthread': 8, # the thread number in the executor process.
          'verbosity': 3
}
print("XGBoost Parameters: \n", params)

sc, spark=create_cntx(executors_per_node,cores_per_executor)
#spark.sparkSession.conf.set('org.apache.spark.example.columnar.enabled', 'True')


from sparkxgb import XGBoostClassifier

path = 'file:///home/xgboost/data/xgboost_4M_float.dataframe.parquet'

df=load_bitdata_arrow_flattern(spark, path)
nRuns = 1
for i in range(0, nRuns):
    model, train_time  = run_train(df, params) #trainingData, params
    print('Completed training the model. Time(sec): ', train_time)

appid = sc.applicationId
 
print('Overall time for {} runs: {}'.format(nRuns, timeit.default_timer() - overall_start_time))
print(appid)

XGBoost Parameters: 
 {'labelCol': 'delinquency_12', 'numRound': 100, 'maxDepth': 8, 'maxLeaves': 256, 'alpha': 0.9, 'eta': 0.1, 'gamma': 0.1, 'subsample': 1.0, 'reg_lambda': 1.0, 'scalePosWeight': 2.0, 'minChildWeight': 30.0, 'treeMethod': 'hist', 'objective': 'reg:squarederror', 'growPolicy': 'lossguide', 'numWorkers': 1, 'nthread': 8, 'verbosity': 3}
executor per node: 1
parallelism: 8
memory: 308976m
offheap:30000m
DataFrame schma:  StructType(List(StructField(interest_rate,FloatType,true),StructField(current_actual_upb,FloatType,true),StructField(loan_age,FloatType,true),StructField(remaining_months_to_legal_maturity,FloatType,true),StructField(adj_remaining_months_to_maturity,FloatType,true),StructField(msa,FloatType,true),StructField(current_loan_delinquency_status,FloatType,true),StructField(foreclosure_costs,FloatType,true),StructField(prop_preservation_and_repair_costs,FloatType,true),StructField(asset_recovery_costs,FloatType,true),StructField(misc_holding_expenses,FloatType

IllegalArgumentException: features does not exist. Available: interest_rate, current_actual_upb, loan_age, remaining_months_to_legal_maturity, adj_remaining_months_to_maturity, msa, current_loan_delinquency_status, foreclosure_costs, prop_preservation_and_repair_costs, asset_recovery_costs, misc_holding_expenses, holding_taxes, net_sale_proceeds, credit_enhancement_proceeds, repurchase_make_whole_proceeds, other_foreclosure_proceeds, non_interest_bearing_upb, principal_forgiveness_upb, foreclosure_principal_write_off_amount, servicer_idx, mod_flag_idx, zero_balance_code_idx, repurchase_make_whole_proceeds_flag_idx, servicing_activity_indicator_idx, delinquency_12, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, coborrow_credit_score, mortgage_insurance_type, datatime, orig_channel_idx, first_home_buyer_idx, loan_purpose_idx, property_type_idx, occupancy_status_idx, property_state_i

In [17]:
sc.stop()

In [18]:
!$SPARK_HOME/sbin/stop-slave.sh
!$SPARK_HOME/sbin/stop-master.sh

stopping org.apache.spark.deploy.worker.Worker
stopping org.apache.spark.deploy.master.Master
